# Stacking ensembling

Vamos a crear un ensamble de tipo stacking. Para ello se utilizaron las siguientes fuentes:
 - https://mlwave.com/kaggle-ensembling-guide/
 - http://blog.kaggle.com/2016/12/27/a-kagglers-guide-to-model-stacking-in-practice/
 - https://github.com/emanuele/kaggle_pbr/blob/master/blend.py

Para ello se desarrollaron las siguientes secciones:
 - [Predictores base](#Predictores-base)
     - [Xgboost](#Xgboost)
     - [Random Forest](#Random-Forest)
     - [AdaBoost](#AdaBoost)
 - [Metafeatures](#Metafeatures)
 - [Predictor Stacking](#Predictor-Stacking)

In [23]:
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from time import time
import xgboost as xgb
import pandas as pd
import os

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

set_de_entrenamiento_testing_y_prediccion = '"{}"'.format(os.path.join(
    current_folder,
    '..',
    'Set de entrenamiento, testing y predicción.ipynb'
))
merge_features = '"{}"'.format(os.path.join(current_folder, '..', 'Features', 'Merge features.ipynb'))

Cargo el df con los features.

In [3]:
pd.options.mode.chained_assignment = None
%run $merge_features

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

In [4]:
assert(df_features.shape[0] == get_clean_df()['person'].unique().shape[0])

Cargo el set de entrenamiento.

In [5]:
%run $set_de_entrenamiento_testing_y_prediccion

labels_with_features = labels.merge(df_features, how='inner', on='person')
train = labels_with_features.drop('label', axis=1)
train_target = labels_with_features['label']

### Predictores base

En esta sección vamos a preparar los predictores base a utilizar. Estos son los mismos que se encuentran en la carpeta *Algoritmos de ML*.

In [6]:
base_predictors = []

#### Xgboost

Nota: vamos a usar XGBRegressor para tener la misma interfaz con el resto de los predictores. Los hiperparámetros tienen distinto nombre, pero producen los mismos resultados. Se puede consultar la documentación de xgboost para encontrar los nombres de los parámetros: https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor. 

In [7]:
param = {
    'objective': 'reg:logistic',
    'colsamble_bylevel': 1,
    'colsamble_bytree': 1,
    'min_child_weight': 1,
    'learning_rate': 0.3,
    'max_delta_step': 5,
    'n_estimators': 25,
    'reg_lambda': 3,
    'max_depth': 9,
    'silent': True,
    'subsample': 1,
    'reg_alpha': 2,
    'gamma': 10
}

base_predictors.append(XGBRegressor(**param))

#### Random Forest

In [8]:
param = {
    'bootstrap': True,
    'max_depth': 9,
    'max_features': 37,
    'min_samples_leaf': 30,
    'min_samples_split': 7,
    'n_estimators': 784
}

base_predictors.append(RandomForestRegressor(**param))

#### AdaBoost

In [9]:
param = {
    'n_estimators': 124,
    'loss': 'linear',
    'learning_rate': 0.07,
    'base_estimator': DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
}

base_predictors.append(AdaBoostRegressor(**param))

### Metafeatures

Ahora vamos a realizar una predicción por cada predictor base, y las vamos a agregar a una copia del set de entrenamiento (*train_meta*).

In [10]:
train_meta = train.copy()
for predictor in base_predictors:
    train_meta[predictor.__class__.__name__] = np.nan

Para realizar las predicciones utilizar cross validation.

In [17]:
%%time
kf = KFold(n_splits=10, shuffle=False)
for train_i, validation_i in kf.split(train):    
    for predictor in base_predictors:
        # como warm_start=False cada vez que llamo fit, el modelo se reinicia
        predictor.fit(train.iloc[train_i], train_target.iloc[train_i]) # train
        train_meta[predictor.__class__.__name__].iloc[validation_i] = predictor.predict(train.iloc[validation_i]) # predict

CPU times: user 12min 45s, sys: 148 ms, total: 12min 45s
Wall time: 12min 45s


In [18]:
train_meta.head(3)

,screen_resolution_height mean,screen_resolution_width mean,screen_resolution_height std,screen_resolution_width std,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,...,Saturday,Sunday,Thursday,Tuesday,Wednesday,tarde,maniana,madrugada,noche,XGBRegressor,RandomForestRegressor,AdaBoostRegressor
person,,,,,,,,,,,,,,,,,,,,,,,,,
0566e9c1,568.0,320.0,0.0,0.0,6.0,3.0,1.0,1.0,15.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1,0,0,0,0.019391,0.018639,0.123133
6ec7ee77,640.0,360.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.111775,0.068184,0.329377
abe7a2fb,640.0,360.0,0.0,0.0,9.0,14.0,1.0,0.0,9.0,0.0,4.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.014011,0.033091,0.143785


### Predictor Stacking

Ahora vamos a entrenar un nuevo modelo utilizando como features las predicciones anteriores (metafeatures). También podemos agregar algunos de los features originales.

Para esto realizamos un Random Search. TODO 

In [19]:
param = {
    'solver': 'liblinear'
}
regr = LogisticRegression(**param)
stack_train = train_meta[[predictor.__class__.__name__ for predictor in base_predictors]]

In [20]:
%%time
scores = cross_val_score(regr, stack_train, train_target, cv=10, scoring='roc_auc')
print("Accuracy: %0.6f (+/- %0.6f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.875340 (+/- 0.024648)
CPU times: user 316 ms, sys: 4 ms, total: 320 ms
Wall time: 320 ms


### Bayesian Optimization

In [21]:
pbounds = {
    'max_depth': (2, 20),
    'eta': (0, 0.3),
    'gamma': (0, 10),
#     'min_child_weight': (1, 5),
#     'max_delta_step': (1, 5),
#     'subsample': (0, 1),
#     'colsample_bytree': (0, 1),
#     'colsample_bylevel': (0, 1),
#     'lambda': (1, 3),
#     'alpha': (0, 2)
}

discrete = ['max_depth'] # parámetros discretos
cv_splits = 10 # cantidad de splits en el cv
num_round = 100 # cantidad máxima de boosts

In [24]:
dtrain = xgb.DMatrix(stack_train, label=train_target)
def cv_score_xgb(**param):
    param['silent'] = 1
    param['objective'] = 'reg:logistic'
    
    # transformo los valores que deben ser discretos
    for d in discrete:
        param[d] = int(param[d])
    
    # hago el cv
    scores = xgb.cv(param, dtrain, nfold=cv_splits, metrics='auc', verbose_eval=False, shuffle=False, stratified=False, num_boost_round=num_round)
    return scores['test-auc-mean'].max()

In [26]:
from bayes_opt import BayesianOptimization

In [27]:
%%time
optimizer = BayesianOptimization(f=cv_score_xgb, pbounds=pbounds)
optimizer.maximize(
    init_points=10,
    n_iter=500,
)

|   iter    |  target   |    eta    |   gamma   | max_depth |
-------------------------------------------------------------
|  1        |  0.8766   |  0.2498   |  9.116    |  10.08    |
|  2        |  0.8768   |  0.07318  |  7.439    |  14.83    |
|  3        |  0.8776   |  0.2973   |  9.086    |  7.108    |
|  4        |  0.8681   |  0.2844   |  4.174    |  17.85    |
|  5        |  0.876    |  0.08289  |  5.276    |  10.42    |
|  6        |  0.8749   |  0.2813   |  3.468    |  6.217    |
|  7        |  0.8758   |  0.1885   |  7.309    |  17.3     |
|  8        |  0.8751   |  0.2117   |  6.247    |  19.28    |
|  9        |  0.8766   |  0.2477   |  8.553    |  16.81    |
|  10       |  0.8781   |  0.06027  |  8.548    |  16.41    |
|  11       |  0.5      |  0.0      |  10.0     |  2.0      |
|  12       |  0.5      |  0.0      |  0.0      |  20.0     |
|  13       |  0.8778   |  0.3      |  0.0      |  2.0      |
|  14       |  0.8684   |  0.3      |  0.0      |  9.116    |
|  15   

|  80       |  0.8769   |  0.3      |  7.843    |  10.36    |
|  81       |  0.8776   |  0.3      |  8.575    |  8.929    |
|  82       |  0.8778   |  0.3      |  10.0     |  3.996    |
|  83       |  0.8778   |  0.3      |  0.9804   |  2.0      |
|  84       |  0.8771   |  0.3      |  10.0     |  14.75    |
|  85       |  0.8768   |  0.3      |  6.55     |  5.168    |
|  86       |  0.8724   |  0.3      |  2.732    |  7.429    |
|  87       |  0.8748   |  0.3      |  6.756    |  18.64    |
|  88       |  0.8746   |  0.3      |  6.701    |  20.0     |
|  89       |  0.8727   |  0.3      |  5.066    |  11.24    |
|  90       |  0.8776   |  0.3      |  9.117    |  4.463    |
|  91       |  0.8778   |  0.3      |  3.512    |  2.0      |
|  92       |  0.8753   |  0.3      |  7.708    |  13.69    |
|  93       |  0.8751   |  0.3      |  6.626    |  12.3     |
|  94       |  0.8712   |  0.3      |  3.708    |  12.4     |
|  95       |  0.876    |  0.3      |  6.064    |  9.764    |
|  96   

|  161      |  0.8622   |  0.3      |  0.8974   |  16.09    |
|  162      |  0.8731   |  0.3      |  4.205    |  11.66    |
|  163      |  0.8772   |  0.3      |  3.679    |  3.811    |
|  164      |  0.868    |  0.3      |  3.6      |  17.03    |
|  165      |  0.8709   |  0.3      |  4.991    |  18.04    |
|  166      |  0.8761   |  0.3      |  4.084    |  5.418    |
|  167      |  0.8767   |  0.3      |  9.279    |  16.43    |
|  168      |  0.8774   |  0.3      |  7.504    |  8.386    |
|  169      |  0.8778   |  0.3      |  9.579    |  3.053    |
|  170      |  0.8779   |  0.3      |  5.696    |  2.0      |
|  171      |  0.8745   |  0.3      |  0.8925   |  5.637    |
|  172      |  0.8753   |  0.3      |  0.0      |  4.537    |
|  173      |  0.8749   |  0.3      |  5.81     |  11.92    |
|  174      |  0.8669   |  0.3      |  1.949    |  12.31    |
|  175      |  0.8678   |  0.3      |  2.974    |  15.81    |
|  176      |  0.8743   |  0.3      |  6.595    |  19.32    |
|  177  

|  242      |  0.8456   |  0.000195 |  8.984    |  16.73    |
|  243      |  0.8771   |  0.3      |  10.0     |  19.11    |
|  244      |  0.8768   |  0.3      |  9.525    |  19.5     |
|  245      |  0.8768   |  0.3      |  9.493    |  18.68    |
|  246      |  0.8759   |  0.3      |  8.42     |  11.84    |
|  247      |  0.8759   |  0.3      |  7.891    |  9.071    |
|  248      |  0.8773   |  0.3      |  6.297    |  4.201    |
|  249      |  0.8707   |  0.3      |  4.783    |  16.23    |
|  250      |  0.8728   |  0.3      |  5.73     |  16.91    |
|  251      |  0.8744   |  0.3      |  7.554    |  17.35    |
|  252      |  0.8647   |  0.3      |  1.862    |  14.92    |
|  253      |  0.8751   |  0.3      |  1.653    |  5.652    |
|  254      |  0.8777   |  0.3      |  8.953    |  6.288    |
|  255      |  0.8732   |  0.3      |  3.096    |  8.116    |
|  256      |  0.8776   |  0.3      |  5.006    |  3.456    |
|  257      |  0.8778   |  0.3      |  1.485    |  2.41     |
|  258  

|  323      |  0.8741   |  0.3      |  6.657    |  15.98    |
|  324      |  0.875    |  0.3      |  6.844    |  8.494    |
|  325      |  0.8714   |  0.3      |  5.098    |  13.74    |
|  326      |  0.876    |  0.3      |  1.634    |  4.31     |
|  327      |  0.8621   |  0.3      |  0.4989   |  17.65    |
|  328      |  0.8763   |  0.3      |  7.339    |  10.92    |
|  329      |  0.8776   |  0.3      |  9.542    |  6.503    |
|  330      |  0.8702   |  0.3      |  3.362    |  15.3     |
|  331      |  0.8779   |  0.3      |  2.866    |  2.0      |
|  332      |  0.8714   |  0.3      |  2.93     |  9.169    |
|  333      |  0.8759   |  0.3      |  8.47     |  13.31    |
|  334      |  0.8712   |  0.3      |  5.341    |  18.52    |
|  335      |  0.8731   |  0.3      |  0.3935   |  6.059    |
|  336      |  0.8768   |  0.3      |  9.331    |  13.58    |
|  337      |  0.8774   |  0.3      |  9.275    |  8.676    |
|  338      |  0.8664   |  0.3      |  0.0      |  10.84    |
|  339  

|  404      |  0.8755   |  0.3      |  8.993    |  14.59    |
|  405      |  0.8621   |  0.3      |  1.302    |  18.76    |
|  406      |  0.8711   |  0.3      |  4.611    |  17.52    |
|  407      |  0.8756   |  0.3      |  6.647    |  12.87    |
|  408      |  0.8774   |  0.3      |  6.784    |  4.607    |
|  409      |  0.8772   |  0.3      |  6.058    |  4.941    |
|  410      |  0.8768   |  0.3      |  9.641    |  12.31    |
|  411      |  0.8714   |  0.3      |  3.203    |  11.0     |
|  412      |  0.8779   |  0.3      |  5.756    |  2.949    |
|  413      |  0.8665   |  0.3      |  2.228    |  14.35    |
|  414      |  0.8734   |  0.3      |  5.015    |  10.34    |
|  415      |  0.8766   |  0.3      |  1.35     |  3.08     |
|  416      |  0.8776   |  0.3      |  7.716    |  6.982    |
|  417      |  0.8774   |  0.3      |  9.626    |  18.11    |
|  418      |  0.8732   |  0.3      |  6.264    |  20.0     |
|  419      |  0.8675   |  0.3      |  0.5117   |  11.38    |
|  420  

|  485      |  0.8778   |  0.3      |  0.2888   |  2.993    |
|  486      |  0.8764   |  0.3      |  8.429    |  14.74    |
|  487      |  0.8775   |  0.3      |  8.726    |  5.817    |
|  488      |  0.87     |  0.3      |  3.904    |  13.09    |
|  489      |  0.8775   |  0.3      |  10.0     |  9.081    |
|  490      |  0.8761   |  0.3      |  7.008    |  9.682    |
|  491      |  0.8742   |  0.3      |  7.135    |  18.91    |
|  492      |  0.8779   |  0.3      |  2.123    |  2.434    |
|  493      |  0.8758   |  0.3      |  2.931    |  5.034    |
|  494      |  0.8733   |  0.3      |  5.966    |  13.5     |
|  495      |  0.877    |  0.3      |  3.863    |  3.11     |
|  496      |  0.8764   |  0.3      |  8.943    |  19.59    |
|  497      |  0.8771   |  0.3      |  10.0     |  19.62    |
|  498      |  0.8768   |  0.3      |  6.895    |  6.325    |
|  499      |  0.8642   |  0.3      |  1.329    |  13.73    |
|  500      |  0.8777   |  0.3      |  9.065    |  3.018    |
|  501  

In [28]:
optimizer.max

{'params': {'eta': 0.3, 'gamma': 9.357962170355707, 'max_depth': 2.0},
 'target': 0.8781863}